In [186]:
#system 'gem install bloomer --user-install'
require 'zlib'
require 'msgpack'
require 'digest/md5'
require 'bloomer/msgpackable'
require 'bloomer'
$source_vocabulary_to_fhir_url = {
    'RXNORM'=> 'http://www.nlm.nih.gov/research/umls/rxnorm',
    'LNC'=> 'http://loinc.org',
    'SNOMEDCT_US'=> 'http://snomed.info/sct',
    'NCI_UCUM'=> 'http://unitsofmeasure.org',
    'CPT'=> 'http://www.ama-assn.org/go/cpt',
    'NDFRT'=> 'http://hl7.org/fhir/ndfrt',
    'CVX'=> 'http://hl7.org/fhir/sid/cvx',
    'DSM-5'=> 'http://hl7.org/fhir/sid/dsm5',
    'ICD9CM'=> 'http://hl7.org/fhir/sid/icd-9-cm',
    'ICD10CM'=> 'http://hl7.org/fhir/sid/icd-10',
    'ICF'=> 'http://hl7.org/fhir/sid/icf-nl',
    'ATC'=> 'http://www.whocc.no/atc',
    'NUCCPT'=> 'http://nucc.org/provider-taxonomy',
    'OMIM'=> 'http://www.omim.org'
}

count = 0
$count_by_source_vocabulary = Hash.new(0)
$all_by_source_vocabulary = Hash.new { |h, k| h[k] = [] }

idx_conso_cui = 
idx_conso_cui = 
idx_termtype = 2
idx_source_vocabulary = 11
idx_source_tty = 12
idx_source_code = 13
idx_display = 14

14

In [ ]:
count =  0
all_codes = []
for infile in Dir["scratch/2018AA-full/2018AA/META/MRCONSO.RRF.*.gz"]
  puts "test", infile
  gz = Zlib::GzipReader.new(open(infile))
  gz.each_line do |line|
    parts = line.split("|")
    # only keep preferred terms
    if parts[idx_termtype] != 'P' 
      next
    end
    puts parts
    if count > 10
      break
    end
    $count_by_source_vocabulary[parts[idx_source_vocabulary]] += 1
    if $source_vocabulary_to_fhir_url.key?(parts[idx_source_vocabulary])
      coding_system = $source_vocabulary_to_fhir_url[parts[idx_source_vocabulary]]
      coding_code = parts[idx_source_code]
      coding_display = parts[idx_display]
      tty = parts[idx_source_tty]
      $all_by_source_vocabulary[parts[idx_source_vocabulary]].push({
        :system => coding_system,
        :code => coding_code,
        :display => coding_display,
        :tty => parts[idx_source_tty]
      })
    end
    count +=1
  end
end


def include_codes_in_filter(vs_array, vocab_key, property, values)
  c=0
  $all_by_source_vocabulary.each do |one_vocab_key, terms|
    next if vocab_key != '*' and vocab_key != one_vocab_key
    terms.each do |coding|
      c += 1    
      if values == "*" or values.include? coding[property]
        vs_array.push(coding)
      end
    end
  end
end

$count_by_source_vocabulary.each do |k,c|
  if $source_vocabulary_to_fhir_url.key?(k)
    puts "#{k}\t\t\t#{c}"
  end
end

In [ ]:
require 'bloomer'

In [ ]:
all_by_source_vocabulary["OK"].push(5)

In [ ]:
all_by_source_vocabulary.each {|t,v| puts(t,v)}

In [ ]:
require 'zlib'

idx_rel_parent_cui = 0
idx_rel_parent_aui = 1
idx_rel_child_cui = 4
idx_rel_child_aui = 5
idx_rel_name = 7
count=0
for infile in Dir["scratch/2018AA-full/2018AA/META/MRREL.RRF.*.gz"]
  puts "load rels from", infile
  gz = Zlib::GzipReader.new(open(infile))
  gz.each_line do |line|
    parts = line.split("|")
    # only keep preferred terms
    count += 1
    if count > 10
      break
    end
    
    puts "#{parts[idx_rel_parent_cui]}/#{parts[idx_rel_parent_aui]} #{parts[idx_rel_name]} #{parts[idx_rel_child_cui]}/#{parts[idx_rel_child_aui]}"

  end
end


In [111]:
require "sqlite3"
db = SQLite3::Database.open "output/umls.sqlite"


#<SQLite3::Database:0x005586f7d34ef0 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=nil, @type_translation=nil, @readonly=false>

In [21]:
db.execute"
  select *
  from MRCONSO
  where
    aui='A2880798'
"

[["C0012634", "ENG", "P", "L1222039", "PF", "S0033544", "N", "A2880798", "179085010", "64572001", "", "SNOMEDCT_US", "PT", "64572001", "Disease", 9, "N", 256, ""]]

In [ ]:
matches = db.execute "
  select c.sab, c.tty, c.code, c.str, r.rela,
  c2.sab, c2.tty, c2.code, c2.str
  from
    MRCONSO c
    join mrrel r on c.aui=r.aui2
    join mrconso c2 on c2.aui=r.aui1
  where 
    c.code='74400008' and c.sab='SNOMEDCT_US' and c.tty='PT' and r.rel='CHD'
"
matches.each {|x| puts x}

In [ ]:
matches = db.execute "
with auis as (with recursive 
  ancestor_of_base(parent_aui) as (
    select c.aui
    from
      MRCONSO c
    where 
      c.code='74400008' and c.sab='SNOMEDCT_US' and c.tty='PT'
    union all 
      select r.aui1 from mrrel r join ancestor_of_base a on a.parent_aui=r.aui2 where r.rel='CHD'
  )
  select distinct parent_aui from ancestor_of_base limit 40
)
select c.aui, c.code,c.sab,c.str from auis
join MRCONSO c on c.aui=auis.parent_aui
order by str
"
matches.each {|x| puts x}

In [ ]:
root_aui = db.execute("
  select c.aui
    from
      MRCONSO c
    where 
      c.code='404684003' and c.sab='SNOMEDCT_US' and c.tty='PT'
")[0][0]

require 'set'
ancestor_pool = Set.new()
ancestor_pool

new_next_ancestors = Set.new([root_aui])

# Generate subsumption hierarchy for SNOMED


In [170]:
class AncestryBuilder
  
  attr_accessor :sab, :system, :parents, :ancestry

  def initialize(sab, db)

    @sab = sab
    @system = $source_vocabulary_to_fhir_url[sab]

    @parents =  Hash.new {|h, k| h[k] = Set.new()}
    @ancestry = Hash.new {|h, k| h[k] = Set.new()}

    rels = db.execute("
      select c1.code, c2.code
      from
        mrrel r
        join mrconso c1 on c1.aui=r.aui1
        join mrconso c2 on c2.aui=r.aui2
      where r.rel='CHD' and r.SAB=?
      ",@sab) do |row|    
      rparent,rchild = row
      @parents[rchild].add(rparent)
    end
    
    puts "Selected #{@sab} rels from db"
    
    aa_cached = {}

    @parents.keys.each do |root_code|
      @ancestry[root_code] = if aa_cached[root_code]
        aa_cached[root_code]
      else
        count=0
        ancestor_pool = Set.new()
        new_next_ancestors = Set.new([root_code])
        until new_next_ancestors.empty? do
          count+=1
          next_ancestors = Set.new()
          new_next_ancestors.each do |code|
            next_ancestors += @parents[code]
          end
          new_next_ancestors = next_ancestors - ancestor_pool
          ancestor_pool += new_next_ancestors
        end
        aa_cached[root_code] = ancestor_pool
        aa_cached[root_code]
      end
    end
    puts "Populated ancestry for #{@sab}"  
  end

  def to_coding(code)
    return {
        :system => @system,
        :code => code}
  end

  def subsumed_by(ancestor_code)
    count = 0
    members = Set.new()
    @ancestry.each do |code, ancestor_set|
      if ancestor_set.include? ancestor_code
        count +=1
        members.add(to_coding(code))
      end    
    end
    return members
  end
end

t1 = Time.now
snomed_hierarchy = AncestryBuilder.new("SNOMEDCT_US", db)
t2 = Time.now
puts t2-t1

Selected SNOMEDCT_US rels from db
Populated ancestry for SNOMEDCT_US
265.794631428


In [59]:
x={5=>1}
if x[5]
  3
else
  2
end

3

In [65]:
snomed_hierarchy.subsumed_by("404684003").take(10)

["236016008", "14053009", "162068007", "300402000", "300403005", "8644006", "271835004", "248490000", "128050000", "65576009"]

In [169]:
class ConceptBuilder
  def initialize(sab, db)
    @sab = sab
    @system = $source_vocabulary_to_fhir_url[sab]
    @db = db
  end
  
  def to_coding(code)
    return {
        :system => @system,
        :code => code}
  end

  def query_types(type_list)
    matching_concepts = Set.new
    type_list.each do |t|
      by_type = @db.execute("select code from MRCONSO where sab=? and tty=?", [@sab, t])
      matching_concepts += by_type.map {|cl| to_coding(cl[0])}
    end
    return matching_concepts
  end
  
  def query_all()
    matching_concepts = Set.new
    by_type = @db.execute("select code from MRCONSO where sab=?", [@sab])
    matching_concepts += by_type.map {|cl| to_coding(cl[0])}
    return matching_concepts
  end
end
  
ConceptBuilder.new("RXNORM", db).query_types(["GPCK", "BPCK"]).take(10)
ConceptBuilder.new("RXNORM", db).query_all().take(10)

[{:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"38"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"211"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"332"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"479"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"756"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"768"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"769"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"1204"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"1552"}, {:system=>"http://www.nlm.nih.gov/research/umls/rxnorm", :code=>"1627"}]

In [190]:
require 'fhir_dstu2_models'

$ancestry_builders = {
  'SNOMEDCT_US' => snomed_hierarchy
  }

def codes_for_valueset(valueset_filename, db)
  
  vs_argonaut_codes_str = open(valueset_filename).read()
  vs_argonaut_codes = FHIR::DSTU2::Json.from_json(vs_argonaut_codes_str)

  vs_set = Set.new
  puts "Num includes: #{vs_argonaut_codes.compose.include.count}"
  apply_filters = lambda do |vs_element, logtag|
    
    puts "include #{vs_element}: #{logtag}"
    required_system = vs_element.system
    raise "don't know system" unless $source_vocabulary_to_fhir_url.values.include?(required_system)

    required_key = ($source_vocabulary_to_fhir_url.select {|k,v| v == required_system}).keys[0]
    puts required_key
    raise "can't handle filter intersection" unless vs_element.filter.count <= 1
    
    if vs_element.filter.count == 0
      return ConceptBuilder.new(required_key, db).query_all()
    else
      filter = vs_element.filter[0]
      if filter.op == "in"
        return ConceptBuilder.new(required_key, db).query_types(filter.value.split(","))
      elsif filter.op == "is-a"
        return $ancestry_builders[required_key].subsumed_by(filter.value)
      end
    end
    puts "filter done with size: #{vs_set.count}"    
  end
  
  vs_argonaut_codes.compose.include.each do |vs_include|
    vs_set += apply_filters.(vs_include, :include)
  end
  
  vs_argonaut_codes.compose.exclude.each do |vs_exclude|
    vs_set -= apply_filters.(vs_exclude, :exclude)
  end
  
  puts vs_set.count
  return vs_set
end


def write_filter_to_file(vs_set, output_file)
  vs_array = vs_set.to_a.map do |v|
      {
        :system => v[:system],
        :code => v[:code],
        }
  end
    
  Zlib::GzipWriter.open(output_file+".json.gz") { |gz| gz.write ({
    "resourceType"=> "ValueSet",
    "expansion"=> vs_array
  }).to_json}

  expected_size = 100
  false_positive_probability = 0.001

  vs_bloom = Bloomer::Scalable.new(expected_size, false_positive_probability)
  vs_array.each do |coding|
    vs_bloom.add "#{coding[:system]}|#{coding[:code]}"
  end
  
  File.open(output_file+".msgpck", "w") do |f|
    f.write(vs_bloom.to_msgpack())
  end
end

[
  "daf-problem.canonical",
  "substance-sct",
  "medication-codes",
  "observation-codes",
  "observation-value-codes",
  "procedure-type"].each do |vs_json_file|
    
    vs_set = codes_for_valueset("./inferno/resources/argonauts/ValueSet-#{vs_json_file}.json", db)
    write_filter_to_file(vs_set, "./output/#{vs_json_file}")
    
end



0

Num includes: 2
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x005587ae80ce98>: include
SNOMEDCT_US
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x005587ae807a38>: include
SNOMEDCT_US
167594
Num includes: 1
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x0055879ba9a100>: include
SNOMEDCT_US
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x0055879ba996b0>: exclude
SNOMEDCT_US
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x0055879ba98cb0>: exclude
SNOMEDCT_US
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x0055879ba982b0>: exclude
SNOMEDCT_US
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x0055879ba97860>: exclude
SNOMEDCT_US
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x0055879ba96e60>: exclude
SNOMEDCT_US
19386
Num includes: 1
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x005586f02e99c0>: include
RXNORM
127519
Num includes: 1
include #<FHIR::DSTU2::ValueSet::Compose::Include:0x0055879cbca808>: include
LNC
209903
Num includes: 1
include #<FHIR::

0

In [ ]:

count = 0
until new_next_ancestors.empty? do
  count+=1
  puts count
  next_ancestors = Set.new()
  new_next_ancestors.each_slice(100) do |auis|
    next_ancestors += db.execute("
          select r.aui1 from mrrel r
          where r.aui2 in ? and rela='inverse_isa'
    ", [auis]).flatten.to_set
  end

  new_next_ancestors = next_ancestors - ancestor_pool
  ancestor_pool += next_ancestors
end
puts ancestor_pool.count

#ancestor_pool.each {|a| puts a}
#[new_next_ancestors, ancestor_pool]

In [ ]:
matches = db.execute "
with recursive 
  child_of_base(child_aui) as (
    select r.aui1
    from
      MRCONSO c
      join mrrel r on c.aui=r.aui2
    where 
      c.code='74400008' and c.sab='SNOMEDCT_US' and c.tty='PT' and r.rel='PAR'
    union all 
      select r.aui1 from mrrel r join ancestor_of_base a on a.parent_aui=r.aui2
  )
select * from child_of_base
limit 10
"
matches.each {|x| puts x}

In [149]:
Set.new([{:a=> 1}]) + (Set.new([{:b=> 1}]))

#<Set: {{:a=>1}, {:b=>1}}>

In [162]:
def a
  5
end

a#to_proc()

5

In [ ]:
db.execute("
  select
  -- c.cui, c.aui, c.code, c.sab, c.str
  *
  from mrconso c
  join  MRREL r on r.aui1=c.aui
  where r.aui2=?
  and rel='PAR'
  limit 1000",
[aui])

In [ ]:
require 'fhir_dstu2_models'
require 'bloomer'


def write_filter_to_file(vs_array, output_file)
  Zlib::GzipWriter.open(output_file+".json.gz") { |gz| gz.write ({
    "resourceType"=> "ValueSet",
    "expansion"=> vs_array.map do |v|
      {
        :system => v[:system],
        :code => v[:code],
        :display => v[:display]
        }
    end
    }).to_json }

  expected_size = 100
  false_positive_probability = 0.0001

  vs_bloom = Bloomer::Scalable.new(expected_size, false_positive_probability)
  vs_array.each do |coding|
    vs_bloom.add "#{coding[:system]}|#{coding[:code]}"
  end
  
  File.open(output_file+".msgpck", "w") do |f|
    f.write(vs_bloom.to_msgpack())
  end
end

def read_relations()
end

def bloom_filter_for_valueset(valueset_filename, output_file)
  vs_argonaut_codes_str = open(valueset_filename).read()
  vs_argonaut_codes = FHIR::DSTU2::Json.from_json(vs_argonaut_codes_str)
  vs_array = []

  vs_argonaut_codes.compose.include.each do |vs_include|
    required_system = vs_include.system
    raise "don't know system" unless $source_vocabulary_to_fhir_url.values.include?(required_system)

    required_key = ($source_vocabulary_to_fhir_url.select {|k,v| v == required_system}).keys[0]
    puts required_key
    raise "can't handle filter intersection" unless vs_include.filter.count <= 1
    if vs_include.filter.count == 0
      filter_property = nil
      filter_values = "*"
    else
      filter = vs_include.filter[0]
      if filter.op == "in"
        filter_property = filter.property.downcase.to_sym
        filter_values = filter.value.split(",")
      else
        puts "backing out for #{output_file} because of op"
        filter_values = "*"
      end
    end 
    puts(filter_property, filter_values)
    include_codes_in_filter(vs_array, required_key, filter_property, filter_values)
  end
  write_filter_to_file(vs_array, output_file)
  puts "#{output_file}: #{vs_array.count}"
end

bloom_filter_for_valueset("./inferno/resources/argonauts/ValueSet-medication-codes.json", "ValueSet-medication-codes")
bloom_filter_for_valueset("./inferno/resources/argonauts/ValueSet-observation-codes.json", "ValueSet-observation-codes")
bloom_filter_for_valueset("./inferno/resources/argonauts/ValueSet-procedure-type.json", "ValueSet-procedure-type")

In [ ]:
vs_array = []
include_codes_in_filter(vs_array, "SNOMEDCT_US", :tty, "*")
puts vs_array.count
write_filter_to_file(vs_array, "sct-us")


vs_array = []
include_codes_in_filter(vs_array, "RXNORM", :tty, "*")
puts vs_array.count
write_filter_to_file(vs_array, "rxnorm-all")


vs_array = []
include_codes_in_filter(vs_array, "*", :tty, "*")
puts vs_array.count
write_filter_to_file(vs_array, "umls-all")

In [ ]:
rehydrated = Bloomer::Scalable.from_msgpack(open("ValueSet-procedure-type.msgpck").read())
puts rehydrated.count, 26**4

chars = 3
('a'*chars..'z'*chars).map do |n|
  if rehydrated.include?("#{n}")
    puts n
    1
  else
    0
  end
end.sum * 1.0 / (26**chars)

In [ ]:
system